In [ ]:
import torch
torch.set_default_dtype(torch.float64)
seq_len = 12
dim = 5 # head_dim
Q = torch.randn((seq_len, dim),requires_grad=True)
K = torch.randn((seq_len, dim),requires_grad=True)
V = torch.randn((seq_len, dim),requires_grad=True)
S = torch.matmul(Q, K.transpose(0, 1)) # flops--> seq_len^2 x dim
S.retain_grad()
P = torch.softmax(S, dim=-1) # flops--> seq_len^2 x 2     (exp 一次，除sum一次)
P.retain_grad()
O_ori = torch.matmul(P, V) # flops--> seq_len^2 x dim
O_ori.retain_grad()
factor =  torch.randn_like(O_ori)
# loss = torch.sum(O_ori * factor)
# loss.backward(retain_graph=True)
O_ori.backward(factor, retain_graph=True)
dO = factor
print(torch.all(O_ori.grad == dO))
M_ori =  torch.max(S,dim=-1)[0]

tensor(True)


In [4]:
block_qo= 2
block_kv = 3
O = torch.zeros_like(Q)
M = torch.zeros(seq_len) # 保存max + ln^l
for block_idx in range(seq_len // block_qo): # 按块遍历 Q
    q = Q[block_idx*block_qo:(block_idx+1)*block_qo, :]
    o = O[block_idx*block_qo:(block_idx+1)*block_qo, :]
    m_i= torch.ones(block_qo)*float('-inf')
    l_i = torch.zeros(block_qo)
    for block_kv_idx in range(seq_len // block_kv): # 按块遍历 K, V
        k = K[block_kv_idx*block_kv:(block_kv_idx+1)*block_kv, :]
        v = V[block_kv_idx*block_kv:(block_kv_idx+1)*block_kv, :]

        # 求局部值
        s =torch.matmul(q, k.transpose(0, 1))
        m_ij = torch.maximum(torch.max(s,dim=-1)[0],m_i) 
        # torch.max 返回最大值[0]和最大值的索引[1]  ,求出包含当前处理bolck_j的局部最大值
        P_hat = torch.exp(s - m_ij[:,None]) # exp(s - max(s))
        l_ij = torch.sum(P_hat, dim=-1)
        alpha = torch.exp(m_i - m_ij)

        # 更新
        o[...] = o * alpha[:,None]
        o[...] += torch.matmul(P_hat, v)
        l_i = alpha*l_i + l_ij # 更新
        m_i = m_ij  #更新


    o[:] /= l_i[:,None]
    M[block_idx*block_qo:(block_idx+1)*block_qo] = m_i + torch.log(l_i)


torch.allclose(O,O_ori),torch.max(O-O_ori),M

(True,
 tensor(5.5511e-16, grad_fn=<MaxBackward1>),
 tensor([4.4392, 2.9770, 5.4098, 3.5339, 3.2394, 2.7365, 3.3584, 4.1289, 4.4451,
         3.5642, 3.7554, 3.9380], grad_fn=<CopySlices>))

$S = QK^T \in \mathbb{R}^{N\times N}$,   
$P = sfotmax(S)\in \mathbb{R}^{N\times N}$,        
$O =PV \in \mathbb{R}^{N\times d} $      

记$\frac{\partial L_{loss}}{\partial O} = dO$

$ dV = P^T dO$,
$ dP = dO V^T$,



$ dP_{ij} = do_i^T v_j$

$$
dP_{i:} =
\begin{bmatrix}
P_{i1} \\ P_{i2} \\ P_{i3} \\ ...\\ P_{ij} \\
\end{bmatrix}
=
\begin{bmatrix}
do_i^T v_1 \\ do_i^T v_2 \\ do_i^T v_3 \\ ...\\ do_i^T v_j \\ 
\end{bmatrix}
=
\begin{bmatrix}
v_1^T do_i \\ v_2^T do_i \\ v_3^T do_i \\ ...\\ v_j^T do_i \\ 
\end{bmatrix}
=
\begin{bmatrix}
v_1^T  \\ v_2^T  \\ v_3^T  \\ ...\\ v_j^T \\ 
\end{bmatrix} \,do_i
$$
<!--  -->
$$ 
\begin{align}
dS_{i:} &= M_{Jocabian}\, dP_{i:} \\
&= (diag(P_{i:}) - P_{i:} P_{i:}^T)dP_{i:}\\    
&= diag(P_{i:})dP_{i:} - P_{i:} P_{i:}^TdP_{i:}\\
&= P_{i:}\circ dP_{i:} - P_{i:}^TdP_{i:}P_{i:} \quad \text{点积结果是标量可以交换}
\end{align}
$$
<!--  -->
$$
D_i = P_{i:}^TdP_{i:} 
= 
\begin{bmatrix}
P_{i1} &  P_{i2} & P_{i3} & ...& P_{ij} 
\end{bmatrix}

\begin{bmatrix}
v_1^T  \\ v_2^T  \\ v_3^T  \\ ...\\ v_j^T \\ 
\end{bmatrix} \,do_i
= o_i^T do_i = do_i^T o_i
$$


In [ ]:
# 该伪码实现参照flashattention v2

block_qo= 2
block_kv = 3

dQ = torch.zeros_like(Q)
dK = torch.zeros_like(K)
dV = torch.zeros_like(V)
D = torch.sum(dO * O,dim=-1)
for block_kv_idx in range(seq_len//block_kv):
    k_j = K[block_kv_idx*block_kv:(block_kv_idx+1)*block_kv,:]#load to SRAM
    v_j = V[block_kv_idx*block_kv:(block_kv_idx+1)*block_kv,:]#load to SRAM
    dk_j = torch.zeros_like(k_j)#init on SRAM
    dv_j = torch.zeros_like(v_j)#init on SRAM
    for block_qo_idx in range(seq_len//block_qo):
        q_i = Q[block_qo_idx*block_qo:(block_qo_idx+1)*block_qo,:] #load to SRAM
        
        o_i = O[block_qo_idx*block_qo:(block_qo_idx+1)*block_qo,:] #load to SRAM
        do_i = dO[block_qo_idx*block_qo:(block_qo_idx+1)*block_qo,:] #load to SRAM
        m_i = M[block_qo_idx*block_qo:(block_qo_idx+1)*block_qo] #load to SRAM
        D_i = D[block_qo_idx*block_qo:(block_qo_idx+1)*block_qo] #load to SRAM

        s_ij = torch.matmul(q_i, k_j.transpose(0, 1))
        p_ij = torch.exp(s_ij-m_i[:,None])

        dv_j += torch.matmul(p_ij.transpose(0, 1), do_i)
        dp_ij = do_i @ v_j.transpose(0, 1)

        ds_ij = p_ij * (dp_ij -D_i[:,None])

        dq_i = dQ[block_qo_idx*block_qo:(block_qo_idx+1)*block_qo,:] #load to SRAM
        dq_i += torch.matmul(ds_ij, k_j)
        dQ[block_qo_idx*block_qo:(block_qo_idx+1)*block_qo,:] = dq_i #write back to HBM

        dk_j += torch.matmul(ds_ij.transpose(0, 1), q_i)

    dK[block_kv_idx*block_kv:(block_kv_idx+1)*block_kv,:] = dk_j #write back to HBM
    dV[block_kv_idx*block_kv:(block_kv_idx+1)*block_kv,:] = dv_j #write back to HBM

torch.allclose(dQ,Q.grad),torch.allclose(dK,K.grad),torch.allclose(dV,V.grad)        
# torch.max(dQ-Q.grad),torch.max(dK-K.grad),torch.max(dV-V.grad)

(True, True, True)

In [ ]:
dV = P.T @ dO  # seq_len x seq_len  @ seq_len x dim   flops--> seq_len^2 x dim
print(torch.allclose(V.grad ,dV))
dP = dO @ V.T # seq_len x dim @ dim x seq_len   flops--> seq_len^2 x dim
print(torch.allclose(P.grad ,dP))
D = torch.sum(dO * O_ori, dim=-1)
dS = P * (dP - D[:, None]) # seq_len x seq_len     flops--> seq_len^2
print(torch.allclose(S.grad ,dS))
dQ = dS @ K  # seq_len x seq_len @ seq_len x dim  flops--> seq_len^2 x dim
print(torch.allclose(Q.grad ,dQ))
dK = dS.T @ Q # seq_len x dim @ dim x seq_len  flops--> seq_len^2 x dim
print(torch.allclose(K.grad ,dK))   

True
torch.Size([12, 5]) torch.Size([12, 5])
True
True
True
True


In [65]:
i = (0,1) # 0~seq_len-1
j = (0,1) # 0~seq_len-1
o_i =  O_ori[i,:]
p_i = P[i,:]
do_i = dO[i,:] 
q_i = Q[i,:]
k_i = K[j,:]
v_i = V[j,:]
s_ij = S[i,j]
# print(do_i.shape,V.shape)
dp_i = do_i @ v_i.T
print(torch.allclose(dp_i,P.grad[i,j]))
# # D_i =  (o_i @ do_i.T) # 此处是错误的，需要把对角线元素取出来
# D_i = (o_i * do_i).sum(dim = -1)
# print(do_i.shape,dp_i.shape,D_i.shape)
# ds_i = p_i*dp_i - D_i[:,None]*p_i
# torch.allclose(ds_i,S.grad[i,:])

False


In [ ]:
#-----------------------------------------------
# i = (0,1) # 0~seq_len-1
# o_i =  O_ori[i,:]
# p_i = P[i,:]
# do_i = dO[i,:] 
# q_i = Q[i,:]
# k_i = K[i,:]
# v_i = V[i,:]
# # print(do_i.shape,V.shape)
# dp_i = do_i @ V.T
# print(torch.allclose(dp_i,P.grad[i,:]))
# # D_i =  (o_i @ do_i.T) # 此处是错误的，需要把对角线元素取出来
# D_i = (o_i * do_i).sum(dim = -1)
# print(do_i.shape,dp_i.shape,D_i.shape)
# ds_i = p_i*dp_i - D_i[:,None]*p_i
# torch.allclose(ds_i,S.grad[i,:])



q_i = Q[i,:]
dq_i = ds_i @ K
print(torch.allclose(dq_i,Q.grad[i,:]))
dK = ds_i.T @ q_i
torch.allclose(dK,K.grad)

True


False

In [ ]:
x = torch.randn((5,2,2))
torch.diag(x)

RuntimeError: diag(): Supports 1D or 2D tensors. Got 3D